In [16]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.agents import initialize_agent

from llama_index import GPTSimpleVectorIndex, GPTTreeIndex, SimpleDirectoryReader

In [22]:
# Recursively build index from Retool's docs repo
reader = SimpleDirectoryReader(
    # this needds to be a relative path?
    input_dir='../../retool-docs/Documentation/v3/docs'
    )
documents = reader.load_data()
# DON'T RUN THIS MULTIPLE TIMES -- THIS IS EXPENSIVE
index_simple = GPTSimpleVectorIndex.from_documents(documents)
# index_tree = GPTTreeIndex.from_documents(documents)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 445459 tokens


In [23]:
index_simple.save_to_disk('retool_docs_simple_index.json')
# index_tree.save_to_disk('retool_docs_tree_index.json')

In [24]:
# DON'T RUN THIS MULTIPLE TIMES -- THIS IS EXPENSIVE
index_tree = GPTTreeIndex.from_documents(documents)

INFO:llama_index.indices.common_tree.base:> Building index from nodes: 247 chunks
INFO:openai:error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 83a2b3acef1237d5e7d8e7d4aa444d65 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
INFO:openai:error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a9257299885c456add87321b09feea78 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False


KeyboardInterrupt: 

In [11]:
# Initialize Langchain tools for the agent, including a tool that queries the index
tools = [
#     Tool(
#         name='github_api',
#         func=lambda question: print(question),
#         description="The GitHub API. Useful when you want to create GitHub Enterprise Apps.",
#         return_direct=True
#     ),
    Tool(
        name='retool_docs',
        func=lambda question: str(index_simple.query(question)),
        description="Retool Documentation. Useful for anwswering questions about Retool. Includes best practices, reference API documnetation, how to guides, and reommendations.",
        return_direct=True
    ),
#     Tool(
#         name='retool_intercom',
#         func=lambda question: str(index.query(question)),
#         description="Retool Documentation. Useful for anwswering questions about Retool. Includes best practices, reference API documnetation, how to guides, and reommendations.",
#         return_direct=True
#     ),
#     Tool(
#         name='retool_slack_channels',
#         func=lambda question: str(index.query(question)),
#         description="Retool Documentation. Useful for anwswering questions about Retool. Includes best practices, reference API documnetation, how to guides, and reommendations.",
#         return_direct=True
#     ),
]

In [12]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm = OpenAI(model_name="gpt-4", temperature=0)
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    agent="conversational-react-description",
    memory=memory
)

In [13]:
agent_chain.run("can i use external libraries in a retool app?")

'Yes, you can use external libraries in a Retool app. You can add external JavaScript and CSS libraries to your Retool app by going to the app settings. In the settings, you\'ll find the "Scripts & Styles" section where you can add links to the external libraries you want to use. Once added, the libraries will be available for use within your app. Keep in mind that adding too many external libraries may affect the performance of your app.'